In [ ]:
from my_import import *
import numpy as np

uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "com-exp.local:5555"
#uri = "mirror-ecat-srv.local:5555"
#uri = "192.168.99.10:5555"
io = ZmsgIO(uri)

_1stL = [11,12,13,14,15,16]
_2ndL = [21,22,23,24,25,26]
_3rdL = [31,32,33,34,35,36]

mk_ids = lambda s,v: [s+x for x in v] 
mk_pitch = lambda s : mk_ids(s, [2,3,5])
mk_yaw = lambda s: mk_ids(s, [1,4,6])
legs_pitch_ids =  mk_pitch(10) + mk_pitch(20) + mk_pitch(30)  
legs_yaw_ids =  mk_yaw(10) + mk_yaw(20)+ mk_yaw(30)  


# all 
robot_ids = _1stL + _2ndL + _3rdL 
#robot_ids = _2ndL
#robot_ids = legs_pitch_ids
#robot_ids = [13]

homing = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "HOMING", "name": "homing", "board_id": 1, "homing_par": {"x": [0,3]}}}

xs = list(range(9))
xs = np.multiply(xs,3)
smooth = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": 1,
                             "smooth_par": {"x": xs,
                                            #"y": [0,-0.5, 0, 0.5, 0, -0.8, 0, 0.8, 0]}
                                            "y": [0,-0.2, 0, 0.2, 0, -0.2, 0, 0.2, 0]}
                            }
         } 

xss = [0,4,8]
xss = np.multiply(xss,3)
smooth_x2 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": 12,
                                "smooth_par": {"x": xss,
                                               "y": [0, -2.25, 0]}
                               }
            } 
smooth_x3 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": 13,
                                "smooth_par": {"x": xss,
                                               "y": [0, -1.75, 0]}
                               }
            } 
smooth_x5 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": 15,
                                "smooth_par": {"x": xss,
                                               #"y": [0, 1.75, 0]}
                                               "y": [0, 1.25, 0]}
                               }
            } 

up_x2 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "up_smooth", "board_id": 12,
                                "smooth_par": {"x": xss,
                                               "y": [0, -math.radians(120), 0]}
                               }
        } 
up_x3 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "up_smooth", "board_id": 13,
                                "smooth_par": {"x": xss,
                                               "y": [0, math.radians(60), 0]}
                               }
        } 
up_x5 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "up_smooth", "board_id": 15,
                                "smooth_par": {"x": xss,
                                               "y": [0, -math.radians(45), 0]}
                               }
        } 

park_x3 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "park_smooth", "board_id": 13,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, math.radians(140)]}
                               }
    }

sup_x2 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "stand_up", "board_id": -1,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, -math.radians(50)]}
                               }
    }
sup_x3 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "stand_up", "board_id": -1,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, -math.radians(30)]}
                               }
    }
sup_x5 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "stand_up", "board_id": -1,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, math.radians(20)]}
                               }
    }


add_homing = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing"]}}

add_smooth = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["smooth","up_smooth"]}}

add_park = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["park_smooth"]}}

add_sup = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["stand_up"]}}

io.debug=0

def reply_cmd(cmd):
    reply = io.doit(cmd)
    yaml_msg = yaml.safe_load(reply['msg'])
    return yaml_msg

status_cmds = {
    'FAN_ON':           0x0026,
    'FAN_OFF':			0x0062,
    'LED_ON':           0x0019,
    'LED_OFF':          0x0091,
    'RELEASE_BRAKE':    0x00BD,
    'ENGAGE_BRAKE':     0x00DB,
}
reply_cmds = {
    'CMD_DONE':         0x7800,
    'CMD_WORKING':		0xD000,
    'CMD_ERROR':		0xAA00,
    'CMD_NOCOND':       0xBB00,
}

def ctrl_status_cmd(ctrl_cmd: int, bid: int):
    msg = reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'ctrl_status_cmd': ctrl_cmd}).set_bid(bid))
    msg = reply_cmd(SdoCmd(rd_sdo=['ctrl_status_cmd_ack'],wr_sdo={}).set_bid(bid))
    print(hex(msg['ctrl_status_cmd_ack']))
    return msg['ctrl_status_cmd_ack'] == ctrl_cmd + reply_cmds['CMD_DONE']

def ctrl_status_cmd_str(ctrl_cmd: str, bid: int):
    return ctrl_status_cmd(status_cmds[ctrl_cmd], bid) 

def sdo_filter(snames:list, sdos:dict):
    return {key:sdos[key] for key in snames}

In [ ]:
reply = io.doit(master_cmd_get_slave_descr)
slave_descr = yaml.safe_load(reply['msg'])
scan_ids = slave_descr.keys()
scan_ids

# Define trajectories

In [ ]:
# define trajectories
# reset trj names ???
[g for g in io.doit4ids(robot_ids,homing)]
[g for g in io.doit4ids(robot_ids,smooth)]
# overwrite "smooth" for pitch(s) 
[g for g in io.doit4ids([12,22,32],smooth_x2)]
[g for g in io.doit4ids([13,23,33],smooth_x3)]
[g for g in io.doit4ids([15,25,35],smooth_x5)]
#
[g for g in io.doit4ids([12,22,32],up_x2)]
[g for g in io.doit4ids([13,23,33],up_x3)]
[g for g in io.doit4ids([15,25,35],up_x5)]
#
[g for g in io.doit4ids([13,23,33],park_x3)]
# stand up 
[g for g in io.doit4ids([12,22,32],sup_x2)]
[g for g in io.doit4ids([13,23,33],sup_x3)]
[g for g in io.doit4ids([15,25,35],sup_x5)]


In [ ]:
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_set_home)]
for iD in robot_ids :
    io.doit(ctrl_cmd_set_home.set_value(0).set_bid(iD))

# Position test

In [ ]:
# start motors in POSITION
[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x3B).set_gains(u'POSITION',220,35,0,0,0))]
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',2000,40,1,0.4,0.01))]


In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', iD)

In [ ]:
reply = io.doit(add_homing)

In [ ]:
reply = io.doit(add_smooth)

In [ ]:
reply = io.doit(add_sup)

In [ ]:
for i in range(3):
    reply = io.doit(add_smooth)
    reply = io.doit(add_sup)
    reply = io.doit(add_homing)
    reply = io.doit(add_park)
    reply = io.doit(add_homing)
    print ("CICLO ", i)
    #time.sleep(1)

In [ ]:
reply = io.doit(add_park)

In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', iD)

In [ ]:
[g for g in io.doit4ids(robot_ids,ctrl_cmd_stop)]

In [ ]:
io.debug=0
gains = {}
gains['type'] = "POSITION"
gains['pos_kp'] = 200
gains['pos_kd'] = 10
gains['tor_kp'] = 0
gains['tor_kd'] = 0
gains['tor_ki'] = 0
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x3B).set_gains(u'POSITION',200,15,0,0,0))]
[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',300,10,1,0.75,0.005))]
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x3B).set_gains(*gains))]

In [ ]:
for iD in robot_ids:
    reply = io.doit(SdoCmd(rd_sdo=['motor_pos','motor_vel','link_pos','fault'],wr_sdo={}).set_bid(iD))
    yaml_msg = yaml.safe_load(reply['msg'])
    print(iD,yaml_msg)

In [ ]:
io.debug=0
for iD in robot_ids:
    reply = io.doit(SdoCmd(rd_sdo=['gain_0','gain_1','gain_2','gain_','gain_4'],wr_sdo={}).set_bid(iD))
    yaml_msg = yaml.safe_load(reply['msg'])
    print(iD,yaml_msg)

In [ ]:
reply = io.doit(SdoInfo(u'SDO_NAME').set_bid(robot_ids[0]))
yaml_msg = yaml.safe_load(reply['msg'])
sdo_names = yaml_msg
sdo_names

# Impedance

In [ ]:
# start motors in IMPEDANCE
# 1st yaws stiff 200 or 70 damp 10
[g for g in io.doit4ids([11,21,31],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',200,10,1,0.75,0.007))]
# other yaws + last pitch stiff 100 or 30 damp 10
[g for g in io.doit4ids([14,16,24,26,34,36]+[15,25,35],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',100,10,1,0.75,0.007))]
# 1st 2nd pitches stiff 700 or 250 damp 10
[g for g in io.doit4ids([12,13,22,23,32,33],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',700,10,1,0.75,0.007))]
# 3rd leg
#[g for g in io.doit4ids(_3rdL,ctrl_cmd_stop)]
#[g for g in io.doit4ids(_3rdL,ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',0,2,1,0.85,0.005))]


In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', iD)

In [ ]:
reply = io.doit(add_homing)

In [ ]:
reply = io.doit(add_smooth)

In [ ]:
reply = io.doit(add_sup)

In [ ]:
for i in range(5):
    reply = io.doit(add_smooth)
    reply = io.doit(add_sup)
    reply = io.doit(add_homing)
    print ("CICLO ", i)

In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', iD)

In [ ]:
[g for g in io.doit4ids(robot_ids,ctrl_cmd_stop)]

In [ ]:
np.multiply(list(range(9)),10)